In [38]:
# import libraries
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import datetime, re, time, itertools, smtplib, ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [39]:
start_date = datetime.datetime(2020,11,5)
days_range = 4
trip_length, min_length, max_length = 10, 7, 12
price_goal = 200
cities = 'DEN-AUS'

In [40]:
start = pd.date_range(start_date, periods=days_range).to_pydatetime().tolist()
end = pd.date_range(start_date + datetime.timedelta(days=trip_length), periods=days_range).to_pydatetime().tolist()

#Get all combinations of dates
all_dates = list(itertools.product(start,end))

In [41]:
departing=[]
returning=[]
for i in range(len(all_dates)):
    if (([x[1] for x in all_dates][i] - [x[0] for x in all_dates][i]).days > min_length) & \
       (([x[1] for x in all_dates][i] - [x[0] for x in all_dates][i]).days < max_length):
            
        departing.append(([x[0] for x in all_dates][i]).strftime('%Y-%m-%d'))
        returning.append(([x[1] for x in all_dates][i]).strftime('%Y-%m-%d'))
    else:
        None

In [42]:
#cleaning dates to make them usable in loop
for i in range(len(departing)):
    departing[i]=re.sub('00:00:00', '', departing [i])
    returning[i]=re.sub('00:00:00', '', returning[i])

In [50]:
data=[]

for i in range(len(departing)):

    page ='https://www.ca.kayak.com/flights/' + cities + '/' + departing[i]+ '/' + returning[i] +'?sort=bestflight_a' 
    driver = webdriver.Chrome(executable_path=r'/Applications/anaconda3/lib/python3.7/site-packages/selenium/webdriver/common/chromedriver')
    driver.get(page)  
    time.sleep(10)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    my_table = soup.find(class_=['price option-text'])
    
    try:
        data.append(my_table.get_text())
        data[i]=int(re.sub('[^0-9]','', data[i]))
        
        if data[i]<price_goal:
            
            #Email info
            sender = 'tytinkdev@gmail.com'
            password = 'Thetink10!'
            port = 465
            receive = 'maddiej149@gmail.com'
           
            #Email content
            msg = MIMEMultipart()
            msg['From'] = sender
            msg['To'] = receive
            msg['Subject'] = 'Hi pie... How does this deal look?'

            body = 'Lmk...' + re.sub('https://www.', '', page)
            msg.attach(MIMEText(body,'plain'))
            text = msg.as_string()

            #Sending email
            context = ssl.create_default_context()
            with smtplib.SMTP_SSL('smtp.gmail.com', port=port, context=context) as server:
                server.login(sender, password)
                server.sendmail(sender, receive, text)
        else:
            None                 
    except:
        data.append('No flights found')

driver.quit()

KeyboardInterrupt: 